# Python Decorators 

### Wrapping a function

Imagine we have some API endpoint. We will simulate the behaviour of that endpoint with the `api_endpoint` function, which will run after a random uniformly distributed delay between 1 and 2 seconds. 

In [1]:
import time 

def api_endpoint():
    time.sleep(1)
    return 'API response'


In our application we will have some kind of function that performs the call and prints the result. 

In [2]:
def call_api():
    result = api_endpoint()
    return result
    
call_api()

'API response'

Now let's imagine that we would like to time the execution of our api call. We can easily do this by using the `time.time` function and then calculating the differece between the moment that the function started and finished running.

In [3]:
def call_api_timed():
    start = time.time()
    result = api_endpoint()
    end = time.time()
    print('seconds:', end - start)
    return result
    
call_api_timed()

seconds: 1.000215768814087


'API response'

Works as expected ! 


### The Python way

In Python we strive to do the minimum that gets the job done. That's the essence of a true Pythonista: no upfront design required, no complicated code necessary. Simply write the most bare bones, quick and dumb program you can that gets the job done and move on. 

However, there are times when keeping things primitive only makes the overall picture increasingly complicated. This is especially true when our application performs lots of repetitve tasks. This is where the magic of Python lies: Python allows you to extend your code in a simple way, without introducing much overhead. For that purpose it offers a number of coding paradigms, one of which we're going to be using today: **decorators**. 

In our example we wrote 3 lines of code within our `call_api` function in order to keep track of how long the api takes to respond. Imagine now that we have multiple different functions that call different API endpoints and we would also like to time them. If we keep doing it the same way as we did earlier, a lot of code would start to get repeated. 

In [4]:
#
# We would have to repeat the same lines of code 
# for all the following functions
# ...
def api_endpoint_2():
    pass
    
def api_endpoint_3():
    pass

To solve this issue, let us define a function that automatically does the timing for us. 

In [5]:
def time_me(api_result):
    start = time.time()
    end = time.time()
    print('seconds:', end - start)
    return api_result

time_me(call_api())

seconds: 1.1920928955078125e-06


'API response'

Awesome ! We can now wrap this timer around any other function and print out the time it took to execute just as well as get the result back. 

Nevertheless, this introduces another problem: long and convoluted lines of code. If we start to chain functions one after another, our code will become increasingly hard to read. 

``` python

first_function(second_function(third_function(...))) 

```

### Decorator syntax

Enter the decorator syntactic sugar. With the Python decorator syntax, we can set a function A to always be executed before another function B by adding an `@` (function A would then be called a decorator) in front of it before function B's definition. 

In [6]:
@function_A
def function_B():
    pass 

NameError: name 'function_A' is not defined

Every time we call `function_B`, `function_A` is executed beforehand. 

A decorator function is a special type of function that takes the function that it's decorating as an the input and returns a wrapper function that actually performs the operation. We would thus have to rewrite our `time_me` function like so:

In [ ]:
def time_me(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print('seconds:', end - start)
        return result
    return wrapper

@time_me
def call_api():
    res = api_endpoint()
    return res

In [ ]:
call_api()

"Can we pass arguments to the decorator itsefl ?" you might ask. The answer is yes ! But we would have to do some additional modifications to allow our decorator to accept arguments. Let's say you would want this function to sometimes print seconds, while other times you would rather have it print milliseconds. 

Don't worry too much about the `@wraps` decorator: it's a simple Python function that ...

In [7]:
from functools import wraps

def time_me(milliseconds=False):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            if milliseconds:
                print('milliseconds:', (end - start)*1000)
            else:
                print('seconds:', end - start)
            return result
        return wrapper
    return decorator

@time_me(milliseconds=True)
def call_api():
    res = api_endpoint()
    return res

In [8]:
call_api()

milliseconds: 1004.0688514709473


'API response'

Success ! 

## Class implementation

Python treats everything as an object. **Everything**. That includes variables, functions, classes, lists, etc. 

Python includes special class syntax that allows a decorator to be expressed as a class. 


In [55]:
class TimeMe:
    
    def __init__(self, milliseconds=False):
        print('__init__')
        self.milliseconds = milliseconds
        
    def __call__(self, func):
        print('__call__ ')
        def decorator(*args, **kwargs):
            # 1. start timer
            start = time.time()

            # 2. execute wrapped function
            result = func(*args, **kwargs)

            # 3. end timer
            end = time.time()
            
            # 4. print result
            if self.milliseconds:
                print('milliseconds:', (end - start)*1000)
            else:
                print('seconds:', end - start)
            return result
        return decorator     

In [56]:
timer = TimeMe(milliseconds=True)

@timer
def some_func(x):
    return 2*x

# 
# or 
# 

@TimeMe(milliseconds=True)
def some_func(x):
    return 2*x

__init__
__call__ 
__init__
__call__ 


In [57]:
some_func(2)

milliseconds: 0.0019073486328125


4